In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import time

In [ ]:
df = pd.read_excel('../dataset/BizPlay-2025_JAN.xlsx', sheet_name='2025년1월 거래데이터')
df

In [5]:
df.isnull().sum()

Transaction Date                             0
Transaction Time                             0
Transaction Amount                           0
Customer Date of Birth                       0
Customer Gender                              0
Customer Age                                 0
Customer Region (City/County/District)       0
Merchant Region (City/County/District)       0
Merchant ID                                  0
Sub-Merchant ID                           9622
Transaction Processing Agency (VAN)          0
Distance                                     0
dtype: int64

In [6]:
df['Sub-Merchant ID'] = df['Sub-Merchant ID'].fillna(0)
df

,Transaction Date,Transaction Time,Transaction Amount,Customer Date of Birth,Customer Gender,Customer Age,Customer Region (City/County/District),Merchant Region (City/County/District),Merchant ID,Sub-Merchant ID,Transaction Processing Agency (VAN),Distance
0,20250101,225951,30500,140929,4,10,41133,ONLINE,F000000001,S220106718,52,0
1,20250101,225950,15500,90921,4,15,41220,41220,F000000018,00917255276,55,0
2,20250101,225936,10500,80912,3,16,41150,41150,F000000018,00913070265,55,0
3,20250101,225917,5800,91111,3,15,41430,41430,F000000018,00912455517,55,0
4,20250101,225904,5000,110224,3,14,41360,41360,F000000018,00919265356,55,0
...,...,...,...,...,...,...,...,...,...,...,...,...
323456,20250131,61432,15580,151031,4,9,41670,41670,F000000018,00922577359,55,0
323457,20250131,60746,6050,71023,3,17,41273,41273,F000000018,00921032828,55,0
323458,20250131,60728,31900,70504,3,17,41173,ONLINE,F000000001,S230127579,52,0
323459,20250131,60711,10300,110420,3,14,41173,41173,F000000018,00979351269,55,0


In [7]:
df.isnull().sum()

Transaction Date                          0
Transaction Time                          0
Transaction Amount                        0
Customer Date of Birth                    0
Customer Gender                           0
Customer Age                              0
Customer Region (City/County/District)    0
Merchant Region (City/County/District)    0
Merchant ID                               0
Sub-Merchant ID                           0
Transaction Processing Agency (VAN)       0
Distance                                  0
dtype: int64

### Use Your Rules to Label Pseudo-Anomalies

In [8]:
outside_district = (
        (df['Customer Region (City/County/District)'].astype(str).str.strip() != df['Merchant Region (City/County/District)'].astype(str).str.strip()) &
        (df['Merchant Region (City/County/District)'] != 'ONLINE')
)
outside_district.sum()  # This tells you how many True rows

26203

In [9]:
df['anomaly'] = 0
df.loc[outside_district, 'anomaly'] = 1

df['anomaly'].value_counts()

anomaly
0    297258
1     26203
Name: count, dtype: int64

In [10]:
parsed_time = pd.to_datetime(df['Transaction Time'], format='%H%M%S').dt.time
rule_kids_after_10pm = (df['Customer Age'] < 8) & (parsed_time >= time(22, 0))
df.loc[rule_kids_after_10pm, 'anomaly'] = 1

df['anomaly'].value_counts()

anomaly
0    296975
1     26486
Name: count, dtype: int64

In [11]:
late_night_mask = (parsed_time >= time(23, 0)) | (parsed_time < time(6, 0))
df.loc[late_night_mask, 'anomaly'] = 1

df['anomaly'].value_counts()

anomaly
0    296963
1     26498
Name: count, dtype: int64

In [12]:
financial_aid_recipients_during_school_term = (parsed_time >= time(11, 0)) & (parsed_time < time(13, 0))
df.loc[financial_aid_recipients_during_school_term, 'anomaly'] = 1

df['anomaly'].value_counts()

anomaly
0    258725
1     64736
Name: count, dtype: int64

In [13]:
from sklearn.preprocessing import LabelEncoder

for col in ['Customer Gender', 'Merchant ID', 'Sub-Merchant ID',
            'Transaction Processing Agency (VAN)']:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

In [14]:
region_cols = ['Customer Region (City/County/District)', 'Merchant Region (City/County/District)']
region_df = df[region_cols].copy()

# Apply shared LabelEncoder
combined_region = pd.concat([
    region_df['Customer Region (City/County/District)'],
    region_df['Merchant Region (City/County/District)']
])

encoder = LabelEncoder()
encoder.fit(combined_region.astype(str))

df['Customer_Encoded'] = encoder.transform(region_df['Customer Region (City/County/District)'].astype(str))
df['Merchant_Encoded'] = encoder.transform(region_df['Merchant Region (City/County/District)'].astype(str))
df


,Transaction Date,Transaction Time,Transaction Amount,Customer Date of Birth,Customer Gender,Customer Age,Customer Region (City/County/District),Merchant Region (City/County/District),Merchant ID,Sub-Merchant ID,Transaction Processing Agency (VAN),Distance,anomaly,Customer_Encoded,Merchant_Encoded
0,20250101,225951,30500,140929,1,10,41133,ONLINE,0,51565,2,0,0,13,56
1,20250101,225950,15500,90921,1,15,41220,41220,1,18979,3,0,0,22,22
2,20250101,225936,10500,80912,0,16,41150,41150,1,12525,3,0,0,15,15
3,20250101,225917,5800,91111,0,15,41430,41430,1,11742,3,0,0,36,36
4,20250101,225904,5000,110224,0,14,41360,41360,1,22529,3,0,0,32,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323456,20250131,61432,15580,151031,1,9,41670,41670,1,28837,3,0,0,50,50
323457,20250131,60746,6050,71023,0,17,41273,41273,1,25911,3,0,0,25,25
323458,20250131,60728,31900,70504,0,17,41173,ONLINE,0,53116,2,0,0,17,56
323459,20250131,60711,10300,110420,0,14,41173,41173,1,36344,3,0,0,17,17


In [15]:
df.to_csv("../dataset/cleaned_BizPlay_2025_JAN.csv", index=False)

In [16]:
df['anomaly'].value_counts()

anomaly
0    258725
1     64736
Name: count, dtype: int64